# Curve Fitting (Regression & Interpolation) and Data Visualization dengan C
Skeleton code untuk melakukan fungsi curve fitting dan visualisasi data dengan input CSV.

In [ ]:
# Import library yang diperlukan dan define constants

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <math.h>

#define MAX_ROWS 100

## 1. Fungsi untuk Membaca Data CSV

In [ ]:
// Membaca data X dan Y dari file CSV
FILE *file = fopen("data.csv", "r");
if (!file) {
    printf("Gagal membuka file.\n");
    return 1;
}

double x_all[MAX_ROWS], y_all[MAX_ROWS];
double x_valid[MAX_ROWS], y_valid[MAX_ROWS];
int n_all = 0, n_valid = 0;
char line[100];

fgets(line, sizeof(line), file); // Lewati header

while (fgets(line, sizeof(line), file)) {
    double x, y;
    char *token = strtok(line, ",");
    x = atof(token);

    token = strtok(NULL, ",");
    if (token == NULL || token[0] == '\n') {
        y = -1; // Nilai Y kosong
    } else {
        y = atof(token);
        x_valid[n_valid] = x;
        y_valid[n_valid] = y;
        n_valid++;
    }

    x_all[n_all] = x;
    y_all[n_all] = y;
    n_all++;
}
fclose(file);

## 2. Fungsi untuk Curve Fitting - Regresi Linear

In [ ]:
void linear_regression(double x[], double y[], int n, double *a, double *b) {
    double sum_x = 0, sum_y = 0, sum_xy = 0, sum_x2 = 0;
    for (int i = 0; i < n; i++) {
        sum_x += x[i];
        sum_y += y[i];
        sum_xy += x[i] * y[i];
        sum_x2 += x[i] * x[i];
    }
    *a = (n * sum_xy - sum_x * sum_y) / (n * sum_x2 - sum_x * sum_x);
    *b = (sum_y - (*a) * sum_x) / n;
}

## 3. Fungsi untuk Curve Fitting - Regresi Polinomial

In [ ]:
void polynomial_regression(double x[], double y[], int n, double *a0, double *a1, double *a2) {
    double sum_x = 0, sum_x2 = 0, sum_x3 = 0, sum_x4 = 0;
    double sum_y = 0, sum_xy = 0, sum_x2y = 0;

    for (int i = 0; i < n; i++) {
        double xi = x[i], yi = y[i];
        double xi2 = xi * xi;
        double xi3 = xi2 * xi;
        double xi4 = xi3 * xi;

        sum_x += xi;
        sum_x2 += xi2;
        sum_x3 += xi3;
        sum_x4 += xi4;

        sum_y += yi;
        sum_xy += xi * yi;
        sum_x2y += xi2 * yi;
    }

    double A[3][4] = {
        {n, sum_x, sum_x2, sum_y},
        {sum_x, sum_x2, sum_x3, sum_xy},
        {sum_x2, sum_x3, sum_x4, sum_x2y}
    };

    for (int i = 0; i < 3; i++) {
        for (int k = i + 1; k < 3; k++) {
            double t = A[k][i] / A[i][i];
            for (int j = 0; j < 4; j++) {
                A[k][j] -= t * A[i][j];
            }
        }
    }

    for (int i = 2; i >= 0; i--) {
        A[i][3] /= A[i][i];
        A[i][i] = 1;
        for (int k = i - 1; k >= 0; k--) {
            double t = A[k][i];
            A[k][3] -= t * A[i][3];
            A[k][i] = 0;
        }
    }

    *a0 = A[0][3];
    *a1 = A[1][3];
    *a2 = A[2][3];
}

## 4. Fungsi untuk Interpolasi

In [ ]:
double interpolate_linear(double x[], double y[], int n, double x_target) {
    for (int i = 0; i < n - 1; i++) {
        if (x_target > x[i] && x_target < x[i + 1]) {
            double slope = (y[i + 1] - y[i]) / (x[i + 1] - x[i]);
            return y[i] + slope * (x_target - x[i]);
        }
    }
    return -9999; 
}

## 5. Fungsi untuk Visualisasi Data

In [ ]:
FILE *fout = fopen("output_linear.dat", "w");
fprintf(fout, "#X\tY_Original\tY_Linear\tY_Polynomial\tY_Interpolasi\n");

for (int i = 0; i < n_all; i++) {
    double x_val = x_all[i];
    double y_val = y_all[i];
    double y_lin = a * x_val + b;
    double y_poly = a2 * x_val * x_val + a1 * x_val + a0;
    double y_interp = (y_val == -1) ? interpolate_linear(x_valid, y_valid, n_valid, x_val) : y_val;

    double y_real = (y_val == -1) ? y_lin : y_val;
    fprintf(fout, "%.2f\t%.2f\t%.2f\t%.2f\t%.2f\n",
            x_val, y_real, y_lin, y_poly, y_interp);
}
fclose(fout);

## 6. Fungsi untuk Menampilkan Statistik Data

In [ ]:
printf("Regresi Linear: y = %.4fx + %.4f\n", a, b);
printf("Regresi Polinomial: y = %.4fx² + %.4fx + %.4f\n", a2, a1, a0);

## 7. Demo Penggunaan

In [ ]:
for (int i = 0; i < n_all; i++) {
    if (y_all[i] == -1) {
        double x_missing = x_all[i];
        double y_linear = a * x_missing + b;
        double y_poly = a2 * x_missing * x_missing + a1 * x_missing + a0;
        double y_interp = interpolate_linear(x_valid, y_valid, n_valid, x_missing);

        printf("\nPrediksi untuk X = %.1f:\n", x_missing);
        printf("  Linear     : %.4f\n", y_linear);
        printf("  Polinomial : %.4f\n", y_poly);
        printf("  Interpolasi: %.4f\n", y_interp);
    }
}

## 8. Kode keseluruhan

In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <math.h>

#define MAX_ROWS 100

// Fungsi regresi linear
void linear_regression(double x[], double y[], int n, double *a, double *b) {
    double sum_x = 0, sum_y = 0, sum_xy = 0, sum_x2 = 0;
    for (int i = 0; i < n; i++) {
        sum_x += x[i];
        sum_y += y[i];
        sum_xy += x[i] * y[i];
        sum_x2 += x[i] * x[i];
    }
    *a = (n * sum_xy - sum_x * sum_y) / (n * sum_x2 - sum_x * sum_x);
    *b = (sum_y - (*a) * sum_x) / n;
}

// Fungsi regresi polinomial derajat 2 
void polynomial_regression(double x[], double y[], int n, double *a0, double *a1, double *a2) {
    double sum_x = 0, sum_x2 = 0, sum_x3 = 0, sum_x4 = 0;
    double sum_y = 0, sum_xy = 0, sum_x2y = 0;

    for (int i = 0; i < n; i++) {
        double xi = x[i], yi = y[i];
        double xi2 = xi * xi;
        double xi3 = xi2 * xi;
        double xi4 = xi3 * xi;

        sum_x += xi;
        sum_x2 += xi2;
        sum_x3 += xi3;
        sum_x4 += xi4;

        sum_y += yi;
        sum_xy += xi * yi;
        sum_x2y += xi2 * yi;
    }

    double A[3][4] = {
        {n, sum_x, sum_x2, sum_y},
        {sum_x, sum_x2, sum_x3, sum_xy},
        {sum_x2, sum_x3, sum_x4, sum_x2y}
    };

    // Eliminasi Gauss
    for (int i = 0; i < 3; i++) {
        for (int k = i + 1; k < 3; k++) {
            double t = A[k][i] / A[i][i];
            for (int j = 0; j < 4; j++) {
                A[k][j] -= t * A[i][j];
            }
        }
    }

    for (int i = 2; i >= 0; i--) {
        A[i][3] /= A[i][i];
        A[i][i] = 1;
        for (int k = i - 1; k >= 0; k--) {
            double t = A[k][i];
            A[k][3] -= t * A[i][3];
            A[k][i] = 0;
        }
    }

    *a0 = A[0][3];
    *a1 = A[1][3];
    *a2 = A[2][3];
}

// Interpolasi linear antara 2 titik terdekat
double interpolate_linear(double x[], double y[], int n, double x_target) {
    for (int i = 0; i < n - 1; i++) {
        if (x_target > x[i] && x_target < x[i + 1]) {
            double slope = (y[i + 1] - y[i]) / (x[i + 1] - x[i]);
            return y[i] + slope * (x_target - x[i]);
        }
    }
    return -9999; 
}

int main() {
    FILE *file = fopen("data.csv", "r");
    if (!file) {
        printf("Gagal membuka file.\n");
        return 1;
    }

    double x_all[MAX_ROWS], y_all[MAX_ROWS];
    double x_valid[MAX_ROWS], y_valid[MAX_ROWS];
    int n_all = 0, n_valid = 0;
    char line[100];

    fgets(line, sizeof(line), file); // Skip header file 

    while (fgets(line, sizeof(line), file)) {
        double x, y;
        char *token = strtok(line, ",");
        x = atof(token);

        token = strtok(NULL, ",");
        if (token == NULL || token[0] == '\n') {
            y = -1;
        } else {
            y = atof(token);
            x_valid[n_valid] = x;
            y_valid[n_valid] = y;
            n_valid++;
        }

        x_all[n_all] = x;
        y_all[n_all] = y;
        n_all++;
    }
    fclose(file);

    // Regresi linear
    double a, b;
    linear_regression(x_valid, y_valid, n_valid, &a, &b);
    printf("Regresi Linear: y = %.4fx + %.4f\n", a, b);

    // Regresi polinomial derajat 2
    double a0, a1, a2;
    polynomial_regression(x_valid, y_valid, n_valid, &a0, &a1, &a2);
    printf("Regresi Polinomial: y = %.4fx² + %.4fx + %.4f\n", a2, a1, a0);

    // Prediksi nilai hilang
    for (int i = 0; i < n_all; i++) {
        if (y_all[i] == -1) {
            double x_missing = x_all[i];

            double y_linear = a * x_missing + b;
            double y_poly = a2 * x_missing * x_missing + a1 * x_missing + a0;
            double y_interp = interpolate_linear(x_valid, y_valid, n_valid, x_missing);

            printf("\nPrediksi untuk X = %.1f:\n", x_missing);
            printf("  Linear     : %.4f\n", y_linear);
            printf("  Polinomial : %.4f\n", y_poly);
            printf("  Interpolasi: %.4f\n", y_interp);
        }
    }

    FILE *fout = fopen("output_linear.dat", "w");
    fprintf(fout, "#X\tY_Original\tY_Linear\tY_Polynomial\tY_Interpolasi\n");

    for (int i = 0; i < n_all; i++) {
        double x_val = x_all[i];
        double y_val = y_all[i];
        double y_lin = a * x_val + b;
        double y_poly = a2 * x_val * x_val + a1 * x_val + a0;
        double y_interp = (y_val == -1) ? interpolate_linear(x_valid, y_valid, n_valid, x_val) : y_val;

        double y_real = (y_val == -1) ? y_lin : y_val;
        fprintf(fout, "%.2f\t%.2f\t%.2f\t%.2f\t%.2f\n",
                x_val,
                y_real,
                y_lin,
                y_poly,
                y_interp
        );
    }
    fclose(fout);

    return 0;
}
